In [93]:
# 다른 코드에서 처리한 데이터를 불러와야하기에 부득이하게 새로운 두개의 코드들 정리
import pandas as pd
crop=pd.read_csv('/content/crop.csv')
weather=pd.read_csv('/content/weather.csv')

In [94]:
#crop 전처리
crop = crop.iloc[0:10]
crop['생육기간'] = crop['생육기간'].str.extract('(\d+)').astype(int)
crop

<ipython-input-94-c9248e495ca3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crop['생육기간'] = crop['생육기간'].str.extract('(\d+)').astype(int)


,작물,최고기온,최저기온,강수량(mm),평균강수량(mm),일조량(hr),생육기간,모?파종(월),재배(월),수확(월)
0,감자,23.0,14.0,75~150,112.5,11.0,4,"2, 3","4, 5",6
1,배추,20.0,15.0,165~200,182.5,7.0,3,"4, 8","5, 6, 9, 10","7, 11"
2,상추,22.0,15.0,150~250,200.0,5.0,2,"4, 8","5, 9","6, 10"
3,토마토,27.0,18.0,200~235,217.5,9.0,3,4,"5, 6",7
4,고구마,28.0,20.0,140~180,160.0,11.0,5,5,"6, 7, 8, 9",10
5,옥수수,30.0,20.0,250~400,325.0,11.0,2,5,6,7
6,대파,22.0,13.0,100~150,125.0,7.0,4,6,"7, 8, 9","10, 11"
7,양파,24.0,13.0,50~85,67.5,13.0,6,10,"11, 12, 1, 2, 3, 4",5
8,무,20.0,15.0,200~300,250.0,6.0,2,9,10,11
9,당근,21.0,18.0,135~200,167.5,6.0,3,8,"9, 10",11


In [95]:
#작물 벡터화하는 과정

# 1. '작물' 열을 제외하고 벡터화할거 추출
columns_to_vectorize = crop[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)','생육기간']]

# 2. 벡터화
vectorized_data = columns_to_vectorize.values

# 3. 데이터 프레임으로 변환
crop_vector_df = pd.DataFrame(vectorized_data, columns=['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)','생육기간'])

# 4. 원래 '작물' 열을 추가
crop_vector_df['작물'] = crop['작물'].values

# 결과 확인
crop_vector_df

,최고기온,최저기온,평균강수량(mm),일조량(hr),생육기간,작물
0,23.0,14.0,112.5,11.0,4.0,감자
1,20.0,15.0,182.5,7.0,3.0,배추
2,22.0,15.0,200.0,5.0,2.0,상추
3,27.0,18.0,217.5,9.0,3.0,토마토
4,28.0,20.0,160.0,11.0,5.0,고구마
5,30.0,20.0,325.0,11.0,2.0,옥수수
6,22.0,13.0,125.0,7.0,4.0,대파
7,24.0,13.0,67.5,13.0,6.0,양파
8,20.0,15.0,250.0,6.0,2.0,무
9,21.0,18.0,167.5,6.0,3.0,당근


In [96]:
#기상청 데이터 벡터화하는 과정

# 1. '작물' 열을 제외하고 벡터화할거 추출
columns_to_vectorize = weather[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']]

# 2. 벡터화
vectorized_data = columns_to_vectorize.values

# 3. 데이터 프레임으로 변환
weather_vector_df = pd.DataFrame(vectorized_data, columns=['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)'])

# 4. 원래 '작물' 열을 추가
weather_vector_df['지점명'] = weather['지점명'].values
weather_vector_df['월'] = weather['월'].values
# 결과 확인
weather_vector_df

,최고기온,최저기온,평균강수량(mm),일조량(hr),지점명,월
0,13.800000,-9.950000,37.216667,5.766111,강릉,1
1,16.800000,-7.450000,32.000000,5.677778,강릉,2
2,23.266667,-1.200000,70.733333,6.180000,강릉,3
3,28.116667,2.400000,85.466667,6.350556,강릉,4
4,33.200000,7.700000,86.216667,6.695556,강릉,5
...,...,...,...,...,...,...
1063,35.950000,16.383333,314.800000,4.395556,함양군,8
1064,31.016667,9.650000,181.083333,3.620556,함양군,9
1065,28.033333,0.216667,99.133333,5.688333,함양군,10
1066,23.733333,-4.800000,33.866667,4.827222,함양군,11


In [156]:
from sklearn.preprocessing import RobustScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

scaler = MinMaxScaler()
crop_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']] = scaler.fit_transform(
    crop_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']]
)
weather_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']] = scaler.transform(
    weather_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']]
)


# 중심 이동: 각 데이터셋에서 중앙값 기준으로 잡기 위해 가져옴!
crop_centered_df = crop_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']] - \
                   crop_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']].median()
weather_centered_df = weather_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']] - \
                      weather_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']].median()

# 모든 작물에 대해 유사도와 거리 계산하여 열 추가
for _, crop_row in crop_vector_df.iterrows():
    crop_name = crop_row['작물']
    crop_vector = crop_row[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']].values
    crop_vector_centered = crop_centered_df.loc[crop_row.name].values

    # 코사인 유사도와 거리 계산
    cosine_similarities = []
    euclidean_distances = []
    centered_euclidean_distances = []

    for _, weather_row in weather_vector_df.iterrows():
        weather_vector = weather_row[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']].values
        weather_vector_centered = weather_centered_df.loc[weather_row.name].values

        # 코사인 유사도 계산
        cosine_sim = cosine_similarity([crop_vector], [weather_vector])[0][0]
        cosine_similarities.append(cosine_sim)

        # 유클리드 거리 계산
        euclidean_dist = euclidean(crop_vector, weather_vector)
        euclidean_distances.append(euclidean_dist)

        # 중심 이동된 유클리드 거리 계산
        centered_euclidean_dist = euclidean(crop_vector_centered, weather_vector_centered)
        centered_euclidean_distances.append(centered_euclidean_dist)

    # 계산 결과를 weather_vector_df에 추가
    weather_vector_df[f'{crop_name}_중심이동_유클리드거리'] = centered_euclidean_distances


In [ ]:
#중심이동만 하는 간소화된 코드

from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean
import pandas as pd
import numpy as np

# 데이터 정규화 (Min-Max Scaler)
scaler = MinMaxScaler()
crop_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']] = scaler.fit_transform(
    crop_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']]
)
weather_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']] = scaler.transform(
    weather_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']]
)

# 중심 이동: 중앙값 기준으로 데이터 이동
crop_centered_df = crop_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']] - \
                   crop_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']].median()
weather_centered_df = weather_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']] - \
                      weather_vector_df[['최고기온', '최저기온', '평균강수량(mm)', '일조량(hr)']].median()

# 중심 이동된 유클리드 거리 계산
for _, crop_row in crop_vector_df.iterrows():
    crop_name = crop_row['작물']
    crop_vector_centered = crop_centered_df.loc[crop_row.name].values

    centered_euclidean_distances = []

    for _, weather_row in weather_vector_df.iterrows():
        weather_vector_centered = weather_centered_df.loc[weather_row.name].values

        # 중심 이동된 유클리드 거리 계산
        centered_euclidean_dist = euclidean(crop_vector_centered, weather_vector_centered)
        centered_euclidean_distances.append(centered_euclidean_dist)

    # 계산 결과를 weather_vector_df에 추가
    weather_vector_df[f'{crop_name}_중심이동_유클리드거리'] = centered_euclidean_distances


In [157]:
weather_vector_df

,최고기온,최저기온,평균강수량(mm),일조량(hr),지점명,월,감자_코사인유사도,배추_코사인유사도,상추_코사인유사도,토마토_코사인유사도,...,감자_중심이동_유클리드거리,배추_중심이동_유클리드거리,상추_중심이동_유클리드거리,토마토_중심이동_유클리드거리,고구마_중심이동_유클리드거리,옥수수_중심이동_유클리드거리,대파_중심이동_유클리드거리,양파_중심이동_유클리드거리,무_중심이동_유클리드거리,당근_중심이동_유클리드거리
0,-0.620000,-3.278571,-0.117605,0.095764,강릉,1,-0.215282,-0.493048,-0.540029,-0.664065,...,2.029851,1.945351,2.106057,2.714461,2.996707,3.208837,1.840207,2.071293,2.002201,2.368851
1,-0.320000,-2.921429,-0.137864,0.084722,강릉,2,-0.191967,-0.507212,-0.529959,-0.632916,...,1.573500,1.489273,1.657181,2.255382,2.535960,2.766484,1.376567,1.639007,1.565315,1.916704
2,0.326667,-2.028571,0.012557,0.147500,강릉,3,-0.045710,-0.444806,-0.396546,-0.438641,...,0.585532,0.445838,0.692360,1.150941,1.437102,1.684092,0.441011,0.836806,0.606533,0.890602
3,0.811667,-1.514286,0.069773,0.168819,강릉,4,0.114539,-0.354760,-0.218261,-0.178307,...,0.482375,0.472104,0.558889,0.450274,0.753232,1.034861,0.586543,0.805829,0.606075,0.539872
4,1.320000,-0.757143,0.072686,0.211944,강릉,5,0.356182,-0.145163,0.088978,0.274075,...,1.252323,1.299509,1.263341,0.534403,0.309221,0.626202,1.428459,1.441909,1.357358,1.008849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,1.595000,0.483333,0.960388,-0.075556,함양군,8,0.407988,0.485760,0.794967,0.817226,...,2.759113,2.602607,2.511267,1.955657,1.846884,1.568392,2.849671,2.979418,2.527842,2.239620
1064,1.101667,-0.478571,0.441100,-0.172431,함양군,9,0.193829,0.022666,0.386965,0.368634,...,1.637100,1.416064,1.325750,0.872369,0.901256,0.850862,1.663748,1.906593,1.367367,1.042118
1065,0.803333,-1.826190,0.122848,0.086042,함양군,10,0.039401,-0.379791,-0.239524,-0.248771,...,0.469024,0.427389,0.476305,0.684471,1.038317,1.231664,0.390662,0.780479,0.527812,0.711156
1066,0.373333,-2.542857,-0.130615,-0.021597,함양군,11,-0.134489,-0.523872,-0.449492,-0.506031,...,0.928912,0.895815,0.999661,1.557989,1.863620,2.099714,0.649399,1.048099,1.002189,1.330193


##이제 모든 작물에 대해서 진행(중심이동 유클리드를 이용하여 적용!)

In [164]:
### 이 코드를 쓰는 이유
### 작물별로 생육기간이 다르기에 그거 연속된 값의 평균을 구함으로써 어떤 연속된 달들이 최적일지 선정하기 위해!
### 이때 1,2,3,4의 경우에는 그냥 되지만 11,12,1도 순환되는 값이기에 이를 처리하기 위한 롤링 처리!


# 각 작물별 생육 기간 설정
crop_periods = {
    "감자": 4,
    "배추": 3,
    "상추": 2,
    "토마토": 3,
    "고구마": 5,
    "옥수수": 2,
    "대파": 4,
    "양파": 6,
    "무": 2,
    "당근": 3
}

# 연속된 n개월 동안 유사도 계산 및 월 정보 추가 (순환 처리)
def calculate_rolling_similarity_with_wrap(df, column, period):
    # 월 정렬
    df = df.sort_values(by="월")

    # 월 순환 처리를 위해 데이터를 이어붙임
    df_extended = pd.concat([df, df], ignore_index=True)

    # n개월 롤링 평균 계산
    rolling_means = []
    rolling_months = []

    for i in range(len(df)):
        # 연속된 n개월을 순환적으로 계산
        window = df_extended.iloc[i:i+period]
        if len(window) == period:
            rolling_means.append(window[column].mean())
            rolling_months.append(list(window["월"]))
        else:
            rolling_means.append(None)
            rolling_months.append(None)

    # 원래 데이터에 연속된 n개월 유사도 및 월 리스트 추가
    df["연속_n개월_유사도"] = rolling_means
    df["연속_n개월_월들"] = rolling_months
    return df[["지점명", "월", "연속_n개월_유사도", "연속_n개월_월들"]]

# 모든 작물의 생육 기간에 따라 계산
results = []

for crop, period in crop_periods.items():
    column_name = f"{crop}_중심이동_유클리드거리"

    # "작물"별로 연속된 n개월 유사도 계산
    result = weather_vector_df.groupby("지점명").apply(
        calculate_rolling_similarity_with_wrap, column=column_name, period=period
    )
    result["작물"] = crop  # 작물 이름 추가
    results.append(result)

# 모든 결과를 병합
final_result = pd.concat(results, ignore_index=True)

# 상위 10개의 유사도 결과를 작물별로 추출
top_results = final_result[final_result["연속_n개월_유사도"].notnull()]
top_results = top_results.groupby("작물").apply(lambda x: x.nsmallest(10, "연속_n개월_유사도")).reset_index(drop=True)

<ipython-input-164-ad8d5efb8261>:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = weather_vector_df.groupby("지점명").apply(
<ipython-input-164-ad8d5efb8261>:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = weather_vector_df.groupby("지점명").apply(
<ipython-input-164-ad8d5efb8261>:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and

In [165]:
top_results

,지점명,월,연속_n개월_유사도,연속_n개월_월들,작물
0,영덕,2,0.860541,"[2, 3, 4, 5]",감자
1,대구,2,0.888449,"[2, 3, 4, 5]",감자
2,경주시,2,0.890151,"[2, 3, 4, 5]",감자
3,울진,2,0.897250,"[2, 3, 4, 5]",감자
4,울산,2,0.897820,"[2, 3, 4, 5]",감자
...,...,...,...,...,...
95,양산시,3,0.677520,"[3, 4, 5]",토마토
96,울진,4,0.680894,"[4, 5, 6]",토마토
97,이천,4,0.684358,"[4, 5, 6]",토마토
98,상주,4,0.691085,"[4, 5, 6]",토마토


In [166]:
from google.colab import files

file_name = "매핑_result.csv"
top_results.to_csv(file_name, index=False, encoding='utf-8-sig')

files.download(file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>